In [8]:
#ORIGINAL CODE

from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_audio_to_duration(file_path, output_path, desired_duration=300, min_silence_len=100, silence_thresh=-30):
    """
    Trims silence from the start and end of an audio file and ensures the resulting file does not exceed the desired duration.
    Prioritizes retaining the latter part of the audio signal if the non-silent duration exceeds the desired length.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed audio will be saved.
    - desired_duration: Maximum duration of the resulting audio file after trimming (in ms).
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # If no nonsilent ranges are detected, the audio is probably all silence.
    if not nonsilent_ranges:
        raise ValueError("No non-silent segments detected in the audio.")
    
    # Only consider the start of the first non-silent segment and the end of the last non-silent segment.
    start, _ = nonsilent_ranges[0]
    _, end = nonsilent_ranges[-1]
    signal = audio[start:end]

    # If signal length exceeds the desired_duration, truncate from the start to prioritize the latter part
    if len(signal) > desired_duration:
        signal = signal[-desired_duration:]
    
    # Save the trimmed audio
    signal.export(output_path, format="wav")

def process_folder(source_folder, output_folder):
    """
    Processes all WAV files in the source folder and saves the trimmed files in the output folder.
    
    Parameters:
    - source_folder: Path to the folder containing WAV files.
    - output_folder: Path to the folder where trimmed files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.wav'):
            source_file_path = os.path.join(source_folder, file_name)
            trimmed_file_name = "trimmed_" + file_name
            output_file_path = os.path.join(output_folder, trimmed_file_name)
            trim_audio_to_duration(source_file_path, output_file_path)

# Usage
input_directory = r"C:\Users\CAN-LAB\Desktop\new"
output_directory = r"C:\Users\CAN-LAB\Desktop\output"
process_folder(input_directory, output_directory)

In [5]:
#VERSION 1
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_and_pad_audio(file_path, output_path, desired_duration=300, min_silence_len=100, silence_thresh=-30):
    """
    Trims silence from the start and end of an audio file and ensures the resulting file is exactly the desired duration (in ms).
    If the audio is shorter than the desired duration, it is padded with silence.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed and padded audio will be saved.
    - desired_duration: Desired duration of the resulting audio file after trimming and padding (in ms).
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # If no nonsilent ranges are detected, the audio is probably all silence.
    if not nonsilent_ranges:
        raise ValueError("No non-silent segments detected in the audio.")
    
    # Only consider the start of the first non-silent segment and the end of the last non-silent segment.
    start, _ = nonsilent_ranges[0]
    _, end = nonsilent_ranges[-1]
    signal = audio[start:end]

    # Calculate the actual duration of the trimmed audio
    actual_duration = len(signal)
    
    # If the actual duration is less than the desired duration, pad with silence
    if actual_duration < desired_duration:
        silence_duration = desired_duration - actual_duration
        silence = AudioSegment.silent(duration=silence_duration)
        signal = signal + silence
    
    # If the signal length exceeds the desired_duration, truncate from the start to prioritize the latter part
    if len(signal) > desired_duration:
        signal = signal[-desired_duration:]
    
    # Save the trimmed and padded audio
    signal.export(output_path, format="wav")

def process_folder(source_folder, output_folder):
    """
    Processes all WAV files in the source folder and saves the trimmed and padded files in the output folder.
    
    Parameters:
    - source_folder: Path to the folder containing WAV files.
    - output_folder: Path to the folder where trimmed and padded files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.wav'):
            source_file_path = os.path.join(source_folder, file_name)
            trimmed_file_name = "trimmed_" + file_name
            output_file_path = os.path.join(output_folder, trimmed_file_name)
            trim_and_pad_audio(source_file_path, output_file_path, desired_duration=300)

# Usage
input_directory = r"C:\Users\CAN-LAB\Desktop\new"
output_directory = r"C:\Users\CAN-LAB\Desktop\output"
process_folder(input_directory, output_directory)


In [9]:
#Final Version

from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import os

def trim_and_pad_audio(file_path, output_path, desired_duration=300, min_silence_len=100, silence_thresh=-30):
    """
    Trims silence from the start and end of an audio file and ensures the resulting file is exactly the desired duration (in ms).
    If the audio is shorter than the desired duration, it is padded with silence evenly at both the beginning and end.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed and padded audio will be saved.
    - desired_duration: Desired duration of the resulting audio file after trimming and padding (in ms).
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # If no nonsilent ranges are detected, the audio is probably all silence.
    if not nonsilent_ranges:
        raise ValueError("No non-silent segments detected in the audio.")
    
    # Only consider the start of the first non-silent segment and the end of the last non-silent segment.
    start, _ = nonsilent_ranges[0]
    _, end = nonsilent_ranges[-1]
    signal = audio[start:end]

    # Calculate the actual duration of the trimmed audio
    actual_duration = len(signal)
    
    # Calculate the required padding on both sides
    padding_duration = desired_duration - actual_duration
    left_padding_duration = padding_duration // 2
    right_padding_duration = padding_duration - left_padding_duration
    
    # Create silence for padding
    left_padding_silence = AudioSegment.silent(duration=left_padding_duration)
    right_padding_silence = AudioSegment.silent(duration=right_padding_duration)
    
    # Add padding to both the beginning and end
    signal = left_padding_silence + signal + right_padding_silence
    
    # Save the trimmed and padded audio
    signal.export(output_path, format="wav")

def process_folder(source_folder, output_folder):
    """
    Processes all WAV files in the source folder and saves the trimmed and padded files in the output folder.
    
    Parameters:
    - source_folder: Path to the folder containing WAV files.
    - output_folder: Path to the folder where trimmed and padded files will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.wav'):
            source_file_path = os.path.join(source_folder, file_name)
            trimmed_file_name = "trimmed_" + file_name
            output_file_path = os.path.join(output_folder, trimmed_file_name)
            trim_and_pad_audio(source_file_path, output_file_path, desired_duration=300)

# Usage
input_directory = r"C:\Users\CAN-LAB\Desktop\new"
output_directory = r"C:\Users\CAN-LAB\Desktop\output"
process_folder(input_directory, output_directory)
